Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Отлично, что доверительный интервал и риск убытков посчитаны верно.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* В работе я оставил несколько советов. Буду рад, если ты учтешь их.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

### <font color='orange'>Общее впечатление (ревью 2)</font>
* После исправлений проект улучшился и теперь он может быть зачтен.
* Для удобства все новые комментарии обозначены фразой "ревью 2".
* Удачи в дальнейшем обучении и следующих работах!

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [71]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [72]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [73]:
geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<b>Создание словаря из датасетов</b>

In [74]:
data_dict = {'reg 1': geo_data_0, 
             'reg 2': geo_data_1, 
             'reg 3': geo_data_2}

<b>Просмотр информации о данных и поиск дубликатов</b>

In [75]:
for key, value in data_dict.items():
    value.info()
    print('Количество дубликатов: ', value.duplicated().sum())
    print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
Количество дубликатов:  0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
Количество дубликатов:  0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null

### Вывод

Данные, содержащие информацию по геологоразведке, представлены в трех файлах, каждый из которых содержит 100000 строк и 5 столбцов. Объем каждого датасета около 4 МБ, пропущенных значений, дубликатов и категориальных признаков нет. Для удобства дальнейшей работы с данными был создан словарь, содержащий датасеты.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены корректно.
</div>

## Обучение и проверка модели

In [76]:
state=np.random.RandomState(123)

In [77]:
results_valid = []

for key, value in data_dict.items():
    features = value.drop(['id', 'product'], axis=1)
    target = value['product']       
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  train_size=0.75, 
                                                                                  test_size=0.25, 
                                                                                  random_state=state)
    model = LinearRegression()
    model.fit(features_train, target_train)   
    predictions = model.predict(features_valid)
    
    stock_average = sum(predictions) / len(predictions)
    rmse = (mean_squared_error(predictions, target_valid))**(0.5)
    
    results_valid.append({'target_valid': target_valid,
                          'predictions': pd.Series(predictions, index=target_valid.index)})
    
    print(key,':', 'Средний запас: ', stock_average)
    print(key,':', 'RMSE: ', rmse)
    print('\n')

reg 1 : Средний запас:  92.54936189116341
reg 1 : RMSE:  37.64786282376176


reg 2 : Средний запас:  69.03253321474074
reg 2 : RMSE:  0.889776104011695


reg 3 : Средний запас:  95.05595470692742
reg 3 : RMSE:  40.017972714679544




### Вывод

Наибольшее среднее значение запаса сырья в регионе 3, но RMSE лучше для региона 2. По значению метрики RMSE можно сделать вывод, что модель отрабатывает более точно для данных региона 2.

<div class="alert alert-block alert-success">
<b>Успех:</b> Модели обучены корректно. Радует, что ты активно используешь циклы. Это помогает избежать дублирования кода, а значит уменьшает вероятность совершить ошибку/опечатку!
</div>

## Подготовка к расчёту прибыли

<b>Сохранение ключевых значений для расчетов в отдельных переменных</b>

In [78]:
POINTS = 500
BEST_POINTS = 200
BUDGET = 10 * 10e8
UNIT_PRIDUCT_PROFIT = 450000
RISK = .025

<div class="alert alert-block alert-info">
<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами.
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Спасибо!
</div>

<b>Рассчет достаточного объёма сырья для безубыточной разработки новой скважины</b>

In [79]:
budget_one_well = BUDGET / BEST_POINTS
min_product_stock = budget_one_well / UNIT_PRIDUCT_PROFIT
print('Минимальный объем ресурсов для разработки новой скважины: ', min_product_stock)

Минимальный объем ресурсов для разработки новой скважины:  111.11111111111111


### Вывод

По результатам расчетов для безубыточной разработки в одной скважине должно содержаться минимум 112 тысяч баррелей нефти, что привышает рассчитанный выше среднестатистический запас.

<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена верно, сравнение проведено!
</div>

## Расчёт прибыли и рисков 

<b>Создание функции для расчёта прибыли по выбранным скважинам и предсказаниям модели</b>

In [80]:
def profit_risk(target, probabilities, count):
    #probs_top = probabilities.sort_values(ascending = False).head(count).index
    #stock_summ = target[probs_top].sum() * unit_product_profit
    
    probs_top = probabilities.sort_values(ascending=False).index
    stock_summ = target[probs_top][:count]
    final_profit = stock_summ.sum() * unit_product_profit
    return final_profit - BUDGET

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Пожалуйста, напечатай размер target[probs_top] во время бутстраппирования, подумай, почему так произошло?
</div>

<div class="alert alert-block alert-warning">
Переписала немного функцию расчета прибыли, возможно, проблема в индексации, но я, если честно, не очень поняла)). Смотрела в нашем тренажере, там есть подобная задача, попробовала там вывести размер target[probs], тоже выходит ошибка: KeyError: 0. Ошибки нет, если в функции sample() я делаю replace=False, то есть отказываюсь от выборок с возвращенными элементами, но мне кажется, это не правильно, может привести к искаженным результатам. 
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Сейчас все верно. Ты права, проблемма в том, что индексы дублируются.
</div>

<b>Подсчет рисков и прибыли для каждого региона с применением техники Bootstrap</b>

In [81]:
for index, dictionary in enumerate(results_valid):
    target_valid = dictionary.get('target_valid')
    predictions = pd.Series(dictionary.get('predictions'), index=target_valid.index)
    
    values = []
    
    for i in range(1000):        
        target_subsample = target_valid.sample(n=POINTS, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        #print(target_subsample[probs_subsample].shape)
        
        values.append(profit_risk(target_subsample, probs_subsample, BEST_POINTS))
    
    values = pd.Series(values)

    mean = values.mean() / 10e6
    lower = values.quantile(RISK) / 10e6
    upper = values.quantile(1 - RISK) / 10e6
    confidence_interval = (lower, upper)
    risk_of_loss = (values < 0).sum() / values.count() * 100

    print('Регион ', index+1,':', 'Средняя прибыль: ', mean.round(2))
    print('Регион ', index+1,':', 'Доверительный интервал 95%: ', confidence_interval)
    print('Регион ', index+1,':', 'Риск убытков: ', risk_of_loss)
    print('\n')

Регион  1 : Средняя прибыль:  50.86
Регион  1 : Доверительный интервал 95%:  (2.000674870515542, 104.31255663184396)
Регион  1 : Риск убытков:  2.3


Регион  2 : Средняя прибыль:  53.74
Регион  2 : Доверительный интервал 95%:  (13.80409948288053, 95.7158372556443)
Регион  2 : Риск убытков:  0.4


Регион  3 : Средняя прибыль:  39.5
Регион  3 : Доверительный интервал 95%:  (-16.65059205621542, 94.70073899384123)
Регион  3 : Риск убытков:  8.5




<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что для подсчета доверительного интервала использовался именно метод квантилей.
</div>

<div class="alert alert-block alert-info">

<b>Совет: </b> Риск можно было найти чуть проще: `(values < 0).mean()`, если объект типа pandas Series или numpy array.
</div>

## Вывод

В ходе работы осуществлено проектирование модели, задача которой - определение региона, разработка месторождений в котором принесет наибольшую прибыль. По результату работы модели можно сделать вывод, что наибольшую прибыль принесет добыча нефти в Регионе 2, при этом риск убытков в данном регионе также является минимальным и составляет порядка 0.4%.

<div class="alert alert-block alert-success">
<b>Успех:</b> С выбором региона согласен.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован